In [1]:
from datasets import load_dataset, DatasetDict, Dataset

In [2]:
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

ModuleNotFoundError: No module named 'peft'

In [3]:
!pip install peft

   ---------------------------------------- 0.0/199.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/199.1 kB ? eta -:--:--
   ------------ --------------------------- 61.4/199.1 kB 1.7 MB/s eta 0:00:01
   ------------------ --------------------- 92.2/199.1 kB 1.1 MB/s eta 0:00:01
   ----------------------- -------------- 122.9/199.1 kB 901.1 kB/s eta 0:00:01
   ----------------------------- -------- 153.6/199.1 kB 833.5 kB/s eta 0:00:01
   -------------------------------------- 199.1/199.1 kB 862.2 kB/s eta 0:00:00


In [4]:
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

ModuleNotFoundError: No module named 'evaluate'

In [5]:
!pip install evaluate

   ---------------------------------------- 0.0/84.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/84.1 kB ? eta -:--:--
   ---------------------------------------- 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [6]:
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [7]:
# # how dataset was generated

# # load imdb data
# imdb_dataset = load_dataset("imdb")

# # define subsample size
# N = 1000 
# # generate indexes for random subsample
# rand_idx = np.random.randint(24999, size=N) 

# # extract train and test data
# x_train = imdb_dataset['train'][rand_idx]['text']
# y_train = imdb_dataset['train'][rand_idx]['label']

# x_test = imdb_dataset['test'][rand_idx]['text']
# y_test = imdb_dataset['test'][rand_idx]['label']

# # create new dataset
# dataset = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train}),
# 

In [8]:
dataset = load_dataset('shawhin/imdb-truncated')

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [10]:
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

0.5

In [12]:
model_checkpoint = 'distilbert-base-uncased'

#deffine label maps
id2label = {0: "Negative", 1:"Positive"}
label2id = {"Nagative":0,"Positive":1}

#generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,num_labels=2,id2label=id2label,label2id=label2id
)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# display architecture 
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [14]:
# create tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,add_prefix_space=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token':'[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
# create tokenize function
def tokenize_function(examples):
    #extract text 
    text = examples['text']
    
    #tokenize and tuncate text 
    tokenizer.truncation_side = "left"
    tokenizer_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True, 
        max_length=512
    )
    
    return tokenizer_inputs

In [17]:
#tokenize training and validation datasets 
tokenized_dataset = dataset.map(tokenize_function,batched=True)
tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [18]:
# create data collator 
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [20]:
# define an evaluation function to pass into trained later
def compute_metrics(p):
    predictions,labels = p 
    predictions = np.argmax(predictions,axis=1)
    
    return {"accuracy": accuracy.compute(predictions=predictions,references=labels)}

In [21]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
It was good. - Positive
Not a fan, don't recommed. - Positive
Better than the first one. - Positive
This is not worth watching even once. - Positive
This one is a pass. - Positive


In [27]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])

In [28]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [29]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9306847223789819


In [30]:
lr = 1e-3
batch_size = 4 
num_epochs = 10

In [31]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [32]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

c:\Users\Wen-Liang\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.3904534876346588, 'eval_accuracy': {'accuracy': 0.876}, 'eval_runtime': 7.3669, 'eval_samples_per_second': 135.743, 'eval_steps_per_second': 33.936, 'epoch': 1.0}
{'loss': 0.4595, 'grad_norm': 6.959514617919922, 'learning_rate': 0.0008, 'epoch': 2.0}


  0%|          | 0/250 [00:00<?, ?it/s]

...

  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.847092866897583, 'eval_accuracy': {'accuracy': 0.886}, 'eval_runtime': 7.37, 'eval_samples_per_second': 135.685, 'eval_steps_per_second': 33.921, 'epoch': 9.0}
{'loss': 0.0054, 'grad_norm': 0.0006044555339030921, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.8639824390411377, 'eval_accuracy': {'accuracy': 0.886}, 'eval_runtime': 7.353, 'eval_samples_per_second': 135.999, 'eval_steps_per_second': 34.0, 'epoch': 10.0}
{'train_runtime': 243.1011, 'train_samples_per_second': 41.135, 'train_steps_per_second': 10.284, 'train_loss': 0.17046285305023193, 'epoch': 10.0}


TrainOutput(global_step=2500, training_loss=0.17046285305023193, metrics={'train_runtime': 243.1011, 'train_samples_per_second': 41.135, 'train_steps_per_second': 10.284, 'train_loss': 0.17046285305023193, 'epoch': 10.0})

In [35]:
model.to('cpu')

print("Trained model predictions:")
print("---------------------------")
for text in text_list:
    inputs = tokenizer.encode(text=text,return_tensors="pt").to("cpu")
    
    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices
    
    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
---------------------------
It was good. - Positive
Not a fan, don't recommed. - Negative
Better than the first one. - Positive
This is not worth watching even once. - Negative
This one is a pass. - Negative
